In [2]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class VanillaEncoder(nn.Module):

    def __init__(self, vocab_size, embedding_size, output_size):
        """Define layers for a vanilla rnn encoder"""
        super(VanillaEncoder, self).__init__()

        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru = nn.GRU(embedding_size, output_size)

    def forward(self, input_seqs, input_lengths, hidden=None):
        embedded = self.embedding(input_seqs)
        packed = pack_padded_sequence(embedded, input_lengths)
        packed_outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = pad_packed_sequence(packed_outputs)
        return outputs, hidden

    def forward_a_sentence(self, inputs, hidden=None):
        """Deprecated, forward 'one' sentence at a time which is bad for gpu utilization"""
        embedded = self.embedding(inputs)
        outputs, hidden = self.gru(embedded, hidden)
        return outputs, hidden

In [2]:
class VanillaDecoder(nn.Module):

    def __init__(self, hidden_size, output_size, max_length, teacher_forcing_ratio, sos_id, use_cuda):
        """Define layers for a vanilla rnn decoder"""
        super(VanillaDecoder, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.log_softmax = nn.LogSoftmax()  # work with NLLLoss = CrossEntropyLoss

        self.max_length = max_length
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.sos_id = sos_id
        self.use_cuda = use_cuda

    def forward_step(self, inputs, hidden):
        # inputs: (time_steps=1, batch_size)
        batch_size = inputs.size(1)
        embedded = self.embedding(inputs)
        embedded.view(1, batch_size, self.hidden_size)  # S = T(1) x B x N
        rnn_output, hidden = self.gru(embedded, hidden)  # S = T(1) x B x H
        rnn_output = rnn_output.squeeze(0)  # squeeze the time dimension
        output = self.log_softmax(self.out(rnn_output))  # S = B x O
        return output, hidden

    def forward(self, context_vector, targets):

        # Prepare variable for decoder on time_step_0
        target_vars, target_lengths = targets
        batch_size = context_vector.size(1)
        decoder_input = Variable(torch.LongTensor([[self.sos_id] * batch_size]))

        # Pass the context vector
        decoder_hidden = context_vector

        max_target_length = max(target_lengths)
        decoder_outputs = Variable(torch.zeros(
            max_target_length,
            batch_size,
            self.output_size
        ))  # (time_steps, batch_size, vocab_size)

        if self.use_cuda:
            decoder_input = decoder_input.cuda()
            decoder_outputs = decoder_outputs.cuda()

        use_teacher_forcing = True if random.random() > self.teacher_forcing_ratio else False

        # Unfold the decoder RNN on the time dimension
        for t in range(max_target_length):
            decoder_outputs_on_t, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs[t] = decoder_outputs_on_t
            if use_teacher_forcing:
                decoder_input = target_vars[t].unsqueeze(0)
            else:
                decoder_input = self._decode_to_index(decoder_outputs_on_t)
            return decoder_outputs, decoder_hidden

In [3]:
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, inputs, targets):
        input_vars, input_lengths = inputs
        encoder_outputs, encoder_hidden = self.encoder.forward(input_vars, input_lengths)
        decoder_outputs, decoder_hidden = self.decoder.forward(context_vector=encoder_hidden, targets=targets)
        return decoder_outputs, decoder_hidden

In [4]:
import config
class Trainer(object):

    def __init__(self, model, data_transformer, learning_rate, use_cuda,
                 checkpoint_name=config.checkpoint_name,
                 teacher_forcing_ratio=config.teacher_forcing_ratio):

        self.model = model

        # record some information about dataset
        self.data_transformer = data_transformer
        self.vocab_size = self.data_transformer.vocab_size
        self.PAD_ID = self.data_transformer.PAD_ID
        self.use_cuda = use_cuda

        # optimizer setting
        self.learning_rate = learning_rate
        self.optimizer= torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.criterion = torch.nn.NLLLoss(ignore_index=self.PAD_ID, size_average=True)

        self.checkpoint_name = checkpoint_name

    def train(self, num_epochs, batch_size, pretrained=False):

        if pretrained:
            self.load_model()

        for epoch in range(0, num_epochs):
            mini_batches = self.data_transformer.mini_batches(batch_size=batch_size)
            for input_batch, target_batch in mini_batches:
                self.optimizer.zero_grad()
                decoder_outputs, decoder_hidden = self.model(input_batch, target_batch)
                # calculate the loss and back prop.
                cur_loss = self.get_loss(decoder_outputs, target_batch[0])
                cur_loss.backward()
                # optimize
                self.optimizer.step()

        self.save_model()

    def get_loss(self, decoder_outputs, targets):
        b = decoder_outputs.size(1)
        t = decoder_outputs.size(0)
        targets = targets.contiguous().view(-1)  # S = (B*T)
        decoder_outputs = decoder_outputs.view(b * t, -1)  # S = (B*T) x V
        return self.criterion(decoder_outputs, targets)

AttributeError: module 'config' has no attribute 'checkpoint_name'

In [42]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [49]:


embedding = nn.Embedding(51,3)
input = torch.LongTensor([[9,5,9,1,2],[4,1,2,2,2],[40,40,40,40,1],[4,4,1,2,2]])
embedded=embedding(input)
print(embedded)


leng = torch.tensor([4,2,5,3])
packed = pack_padded_sequence(embedded, leng, batch_first=True, enforce_sorted=False)
print('\n\n')
print(packed)

tensor([[[ 1.0077,  0.3788, -1.3240],
         [-0.6990, -0.7185,  0.5279],
         [ 1.0077,  0.3788, -1.3240],
         [ 0.8759,  0.2679, -1.5691],
         [-0.0909,  0.3856,  1.0976]],

        [[-0.0590, -0.6038,  0.7650],
         [ 0.8759,  0.2679, -1.5691],
         [-0.0909,  0.3856,  1.0976],
         [-0.0909,  0.3856,  1.0976],
         [-0.0909,  0.3856,  1.0976]],

        [[ 0.0566,  1.4179,  0.0086],
         [ 0.0566,  1.4179,  0.0086],
         [ 0.0566,  1.4179,  0.0086],
         [ 0.0566,  1.4179,  0.0086],
         [ 0.8759,  0.2679, -1.5691]],

        [[-0.0590, -0.6038,  0.7650],
         [-0.0590, -0.6038,  0.7650],
         [ 0.8759,  0.2679, -1.5691],
         [-0.0909,  0.3856,  1.0976],
         [-0.0909,  0.3856,  1.0976]]], grad_fn=<EmbeddingBackward>)



PackedSequence(data=tensor([[ 0.0566,  1.4179,  0.0086],
        [ 1.0077,  0.3788, -1.3240],
        [-0.0590, -0.6038,  0.7650],
        [-0.0590, -0.6038,  0.7650],
        [ 0.0566,  1.4179,  0.00

In [36]:
gru = nn.GRU(3, 3)

In [34]:
packed_outputs, hidden=gru(packed, None)
print(packed_outputs)
print('---------------')
print(hidden)

PackedSequence(data=tensor([[-8.8804e-03, -2.2964e-01, -1.0186e-01],
        [-5.2256e-01,  3.4638e-01,  6.6066e-02],
        [-2.0985e-01, -3.9370e-01, -7.8117e-03],
        [-2.0985e-01, -3.9370e-01, -7.8117e-03],
        [-2.5305e-02, -3.2686e-01, -1.7162e-01],
        [-2.6614e-01, -4.6779e-01,  5.1146e-02],
        [-3.5545e-01, -4.8351e-01,  2.8765e-04],
        [-9.9764e-02, -7.7682e-01,  3.4367e-01],
        [-3.6792e-02, -3.6453e-01, -2.1622e-01],
        [-6.2072e-01,  2.0237e-01,  1.2637e-01],
        [-2.3836e-01, -7.9580e-01,  3.6030e-01],
        [-4.2640e-02, -3.7810e-01, -2.4370e-01],
        [-4.1420e-01, -6.4108e-01,  4.0831e-01],
        [ 5.6516e-02, -7.7115e-01,  2.6235e-01]], grad_fn=<CatBackward>), batch_sizes=tensor([4, 4, 3, 2, 1]), sorted_indices=tensor([2, 0, 3, 1]), unsorted_indices=tensor([1, 3, 0, 2]))
---------------
tensor([[[-0.4142, -0.6411,  0.4083],
         [-0.0998, -0.7768,  0.3437],
         [ 0.0565, -0.7712,  0.2624],
         [-0.2384, -0.7958